# Adversarial Simulator for Question Answering

In [ ]:
from multiprocessing import freeze_support
import os
import asyncio
import logging
from azure.ai.generative.evaluate import evaluate
import json
from azure.ai.generative.synthetic.simulator import Simulator
from azure.ai.resources.client import AIClient
from azure.identity import InteractiveBrowserCredential, DefaultAzureCredential
from azure.ai.resources.entities import AzureOpenAIModelConfiguration
import numpy as np
from openai import AsyncAzureOpenAI

sub = ""
rg = ""
project_name = ""
oai_client = AsyncAzureOpenAI(
    api_key = "",
    azure_endpoint = "",
    api_version = "2023-12-01-preview"
)


## Simulate the data

In [ ]:
async_oai_chat_completion_fn = oai_client.chat.completions.create
client = AIClient(
    subscription_id=sub,
    resource_group_name=rg,
    project_name=project_name,
    credential=DefaultAzureCredential()
)

simulator = Simulator.from_fn(
    fn=async_oai_chat_completion_fn,
    ai_client=client,
    model="gpt-4",
    max_tokens=300
)
adv_template = Simulator.get_template("adv_qa")

## Run the simulator

In [ ]:
outputs = await simulator.simulate_async(
    adv_template,
    max_conversation_turns=2,
    simulation_result_limit=3,
    api_call_delay_sec=10
)

## Print the outputs

In [ ]:
import json
for line in outputs:
    print(json.dumps(line, indent=2))

## Store outputs in file for eval

In [ ]:
eval_name = "conv-oai-qa-harm-eval"
file_name = f"\{eval_name}.jsonl"
jsonl_object = outputs.to_eval_qa_json_lines()
with open(file_name, "w") as f:
    f.write(jsonl_object)

## Run evaluate

In [ ]:
tracking_uri = client.tracking_uri
connection = client.connections.get("Default_AzureOpenAI")
config = AzureOpenAIModelConfiguration.from_connection(
    connection, 
    model_name="gpt-4",
    deployment_name="gpt-4"
)
result = evaluate(
    evaluation_name=eval_name, 
    data=os.path.join(os.getcwd() + f"\{eval_name}.jsonl"),
    task_type="qa",
    metrics_list=["hate_fairness","violence","self_harm", "sexual", "gpt_groundedness", "gpt_relevance", "gpt_coherence", "gpt_fluency", "gpt_similarity"],
    model_config=config,
    data_mapping={
        "questions": "question",
        "answer": "answer",
    },
    tracking_uri=tracking_uri,
    output_path=os.getcwd() + "/downloaded_artifacts/remote"
)
print(result)
print(result.metrics_summary) # will print the defect rate for each content harm
print("Studio URL") 
print(result.studio_url)